In [ ]:
pip install ultralytics

In [2]:
from ultralytics import YOLO
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = YOLO("yolov8n.pt")  # official, guaranteed to work
model.to(device)
model.eval()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [3]:
import cv2
import torch
import numpy as np

DEFECT_CLASSES = {
    0: "missing_component",
    1: "misalignment",
    2: "scratch",
    3: "discoloration"
}

def compute_severity(box, image_area):
    x1, y1, x2, y2 = box
    area = (x2 - x1) * (y2 - y1)
    ratio = area / image_area

    if ratio < 0.01:
        return "low"
    elif ratio < 0.05:
        return "medium"
    else:
        return "high"

def inspect_pcb(image_path, model, device):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    image_area = h * w

    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_tensor = torch.tensor(img_rgb / 255.0, dtype=torch.float32)\
                     .permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)[0]

    results = []

    for box, label, score in zip(
        output["boxes"].cpu().numpy(),
        output["labels"].cpu().numpy(),
        output["scores"].cpu().numpy()
    ):
        if score < 0.5:
            continue

        x1, y1, x2, y2 = map(int, box)
        cx, cy = (x1 + x2)//2, (y1 + y2)//2

        severity = compute_severity((x1, y1, x2, y2), image_area)

        results.append({
            "defect_type": DEFECT_CLASSES[label],
            "confidence": float(score),
            "center": [cx, cy],
            "severity": severity
        })

        cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 2)
        cv2.putText(
            image,
            f"{DEFECT_CLASSES[label]} | {severity}",
            (x1, y1-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0,0,255),
            2
        )

    return image, results


In [4]:
import cv2
import numpy as np

def compute_severity(box, image_area):
    x1, y1, x2, y2 = box
    area = (x2 - x1) * (y2 - y1)
    ratio = area / image_area

    if ratio < 0.01:
        return "low"
    elif ratio < 0.05:
        return "medium"
    else:
        return "high"

def inspect_pcb(image_path, model, device):
    # Read image (DO NOT convert to tensor)
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    image_area = h * w

    # Let YOLO handle resizing internally
    results = model.predict(
        source=image,
        conf=0.5,
        device=device,
        verbose=False
    )[0]

    output_data = []

    if results.boxes is None:
        return image, output_data

    for box, cls, conf in zip(
        results.boxes.xyxy.cpu().numpy(),
        results.boxes.cls.cpu().numpy(),
        results.boxes.conf.cpu().numpy()
    ):
        x1, y1, x2, y2 = map(int, box)
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        severity = compute_severity((x1, y1, x2, y2), image_area)
        defect_name = results.names[int(cls)]

        output_data.append({
            "defect_type": defect_name,
            "confidence": float(conf),
            "bounding_box": [x1, y1, x2, y2],
            "center": [cx, cy],
            "severity": severity
        })

        # Draw bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(
            image,
            f"{defect_name} | {severity}",
            (x1, max(y1 - 10, 10)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 255),
            2
        )

    return image, output_data


In [5]:
import cv2
import json

image_path = "/content/image_defected_car.jpg"

annotated_img, output = inspect_pcb(image_path, model, device)

cv2.imwrite(
    "/content/image_defected_car.jpg",
    annotated_img
)

with open("output.json", "w") as f:
    json.dump(output, f, indent=2)

print("Inspection Results:")
print(json.dumps(output, indent=2))


Inspection Results:
[
  {
    "defect_type": "car",
    "confidence": 0.8177004456520081,
    "bounding_box": [
      56,
      103,
      534,
      261
    ],
    "center": [
      295,
      182
    ],
    "severity": "high"
  }
]
